In [1]:
import numpy as np
import pandas as pd 
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import re
import nltk


In [2]:
products = pd.read_csv(r'.\Recommeder\Data\flipkart_com-ecommerce_sample.csv')
print(products.shape ,len(products['product_name'].unique()),len(products['uniq_id'].unique()))
# products.head()
transactions_path = r".\Recommeder\Data\transaction.csv"

transaction_table = pd.read_csv(transactions_path)

products['description'] = products['description'].fillna('')
corpus = products['product_name']
corpus = corpus.tolist()
# print(corpus[0])
corpus = [x.split() for x in corpus]
print(corpus[0])

(20000, 15) 12676 20000
['Alisha', 'Solid', "Women's", 'Cycling', 'Shorts']


In [3]:


corpus = [re.sub(r'\W', ' ', str(x)) for x in corpus]
corpus = [re.sub(r'\s+[a-zA-Z]\s+', ' ', x) for x in corpus]
corpus = [re.sub(r'\^[a-zA-Z]\s+', ' ', x) for x in corpus]
corpus = [re.sub(r'\s+', ' ', x, flags=re.I) for x in corpus]
corpus = [x.lower() for x in corpus]



In [4]:
pidToidx = pd.Series(products.index,index=products['uniq_id']).drop_duplicates()
pidToProductSeries = pd.Series(products['product_name'].values,index=products['uniq_id']).to_dict()


In [ ]:
# save pidToidx


In [11]:
new_transaction_table = transaction_table.groupby('UID')['PID'].apply(list).reset_index(name='sequence')

In [5]:
model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1, workers=4, sg=0)
model.train(corpus, total_examples=len(corpus), epochs=10)

(1579008, 8220360)

In [6]:
def storeEmbeddings(model , corpus):
    embeddings = []
    for key , product in enumerate(corpus):
        featureVec = np.zeros(100,dtype='float32')
        nwords = 0
        for word in product:
            if word in model.wv:
                nwords += 1
                featureVec = np.add(featureVec,model.wv[word])
        if nwords > 0:
            featureVec = np.divide(featureVec,nwords)
        embeddings.append({'product':product,'vector':featureVec})
    return embeddings

embeddings = storeEmbeddings( model , corpus )


In [7]:
embeddings = pd.DataFrame(embeddings , columns=['product','vector'])

In [39]:
# save embeddings in pickle file
embeddings.to_pickle(r'embeddings.pkl')


In [8]:
# wordsinVocab

def reccomendProductsw2vec(products_bought , embeddings , topK):
    print(products_bought)
    product_embeddings = []
    for idx in products_bought:
        product_embeddings.append(embeddings.iloc[idx]['vector'])
    
    product_embeddings = np.array(product_embeddings)
    avg_embeddings = np.mean(product_embeddings , axis=0)
    similarity_scores = []
    for idx , row in embeddings.iterrows():
        score = cosine_similarity([avg_embeddings],[row['vector']])
        similarity_scores.append((idx,score[0][0]))
        
    
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    product_names = []
    last_score = 0
    for i in similarity_scores:
        if last_score == 0 or abs(last_score - i[1]) >= 0.01*last_score:
            product_names.append(products.iloc[i[0]]['product_name'])
            last_score = i[1]
            if len(product_names) == topK:
                break
            
        
    return product_names


In [12]:
userId = "2e112284-8013-430d-b784-8f1808dd4e76"

pastPurchases = new_transaction_table[new_transaction_table['UID'] == userId]['sequence'].values

# print(pastPurchases)
pastPurchasesIdx = []
for purchase in pastPurchases:
    pastPurchasesIdx.append(pidToidx[purchase].to_list())
    

pastPurchasesIdx = pastPurchasesIdx[0]
print("Past purchases: ", len(pastPurchasesIdx))
# Print name of products bought by user
for i in pastPurchasesIdx:
    print(products.iloc[i]['product_name'])



Past purchases:  13
Silver Kartz Beach Sky Blue Wayfarer, Rectangular Sunglasses
Joy Living Garden Plant Container Set
Delberto Boots
NEWGEN TECH EO-HS3303 55 Wired Headset
Playboy New York Combo Set
Shezone Women Wedges
Floret Women's Bra
Sahara Q Shop Kadhai 1.5 L
Jewel Paradise Brass Plated Brass Necklace
Wild Stone Deodorant No-22 Combo Set
Voylla Yellow Gold Plated Alloy Necklace
Golden Peacock Crystal Alloy Necklace
Voylla Artificial Beaded Glossy Yellow Gold Plated Alloy Necklace


In [13]:

reccomdations = []
# Past Purchases
pastPurchases = new_transaction_table[new_transaction_table['UID'] == userId]['sequence'].values

# Print name of products bought by user
pastProductNames = []
for i in pastPurchases[0]:
    pastProductNames.append(pidToProductSeries[i])

print("Products bought by user: ", len(pastProductNames))

print( pastProductNames )



print("\nTop Recommended products are: \n")
reccomdations = reccomendProductsw2vec(pastPurchasesIdx ,embeddings  , 20)
print("Product Recommended to user: ", len(reccomdations))
print(reccomdations , end = '\n\n')



Products bought by user:  13
['Silver Kartz Beach Sky Blue Wayfarer, Rectangular Sunglasses', 'Joy Living Garden Plant Container Set', 'Delberto Boots', 'NEWGEN TECH EO-HS3303 55 Wired Headset', 'Playboy New York Combo Set', 'Shezone Women Wedges', "Floret Women's Bra", 'Sahara Q Shop Kadhai 1.5 L', 'Jewel Paradise Brass Plated Brass Necklace', 'Wild Stone Deodorant No-22 Combo Set', 'Voylla Yellow Gold Plated Alloy Necklace', 'Golden Peacock Crystal Alloy Necklace', 'Voylla Artificial Beaded Glossy Yellow Gold Plated Alloy Necklace']

Top Recommended products are: 

Product Recommended to user:  20
['KASCN UNIVERSAL ARMY TYPE ONE SIDED BAG FOR ALL MOTORCYCLES One-side Green Fabric Motorbike Saddlebag', "Orange Plum Women's Solid Casual Shirt", "Besiva Women's Solid Formal Shirt", "Lord's Antique Gold Women's Peeptoe Heels Women Heels", 'Scalabedding Cotton Solid King sized Double Bedsheet', "Wrangler Solid Men's Round Neck T-Shirt", 'Juhi Malhotra Ceramic Choker', 'Jums Cotton Bath To